In [1]:
import pandana
import osmnx as ox
import geopandas as gpd
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import geoalchemy2
import numpy as np
import networkx as nx

c:\Users\apikuleva\Anaconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\Users\apikuleva\Anaconda3\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.

In [3]:
# функция для чтения того, что получили в запросе
def select_pg(sql):
    return pd.read_sql(sql, engine)

engine = create_engine('postgresql+psycopg2://kb_geo:kYZQK90yvE8aNi54ELINc0yJ1gu6wo7h@\
db03.cluster.strlk.ru/kb_graph')

In [6]:
node_graph = select_pg('select osmid, \
                       st_x(st_transform(geom,4326)) as lng, \
                        st_y(st_transform(geom,4326)) as lat \
                       from baikal_nodes')

edge_graph = select_pg('select id, source, target, lenght from baikal_edges')

In [8]:
#берем только те ребра, которые совпадают с id узлов 
edge_graph = edge_graph[edge_graph['source'].isin(node_graph['osmid']) & edge_graph['target'].isin(node_graph['osmid'])]
node_graph.set_index('osmid', inplace= True)

In [9]:
edge_graph['distant'] = edge_graph['lenght'].round()

In [12]:
# создаем экземпляр графа
G = nx.from_pandas_edgelist(edge_graph, 'source', 'target', edge_attr='distant')

In [13]:
# Проверка связности графа
is_connected = nx.is_connected(G)

if is_connected:
    print("Граф связный.")
else:
    print("Граф несвязный.")

Граф несвязный.


In [14]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

# Создание пустого датафрейма
combined_df_bridge = pd.DataFrame(columns=['source', 'target', 'Subgraph'])

# Объединение датафреймов в один
for i, subgraph in enumerate(S):
    # Преобразование подграфа в датафрейм
    df = pd.DataFrame(list(subgraph.edges()), columns=['source', 'target'])
    # Добавление столбца с идентификатором подграфа
    df['Subgraph'] = i
    # Объединение с общим датафреймом
    combined_df_bridge = pd.concat([combined_df_bridge, df], axis=0, ignore_index=True)


In [17]:
combined_df_bridge_filter = combined_df_bridge['Subgraph'].value_counts().reset_index().rename(columns={"index": "subgraph", "Subgraph": "count"})\
[combined_df_bridge['Subgraph'].value_counts().reset_index().rename(columns={"index": "subgraph", "Subgraph": "count"})['count'] > 1000]


In [18]:
combined_df_bridge_filter

,subgraph,count
0,0,323741
1,7,2251


In [19]:
combined_df_bridge_filter = combined_df_bridge.merge(combined_df_bridge_filter.head(1), left_on = 'Subgraph', right_on = 'subgraph')

In [25]:
edge_graph

,id,source,target,lenght,distant
0,661757,648761645,648761618,2100.000000,2100.0
1,661758,648761618,648761645,2100.000000,2100.0
2,229,3241855285,3241855291,65.987056,66.0
3,297,3248470605,6842042995,152.852239,153.0
4,365,3248470684,3248470687,190.562359,191.0
...,...,...,...,...,...
661753,661750,7618416588,5085327820,4944.493460,4944.0
661754,661751,7618416588,5085327820,4137.073546,4137.0
661755,661752,7618416588,5085328003,1752.552589,1753.0
661756,661753,7618416588,5085328003,1631.361241,1631.0


In [23]:
road = combined_df_bridge_filter.merge(edge_graph, left_on = ['source', 'target'], right_on = ['source', 'target'])
# road = gpd.GeoDataFrame(road, geometry = road.geometry, crs="EPSG:3857" )
# road['Subgraph'] = road['Subgraph'].astype(int)
# road[['source', 'target', 'geometry', 'Subgraph']].to_file('edge_graph_connected.geojson')

In [26]:
road.to_sql('baikal_edge_graph_connected', engine)

160